In [2]:
# real time MM , extraction of image , text etc from documents, summarizer 

In [ ]:
# !pip show transformers accelerate torch torchvision
# %pip install sentencepiece safetensors



Name: accelerate
Version: 1.5.0
Summary: Accelerate
Home-page: https://github.com/huggingface/accelerate
Author: The HuggingFace team
Author-email: zach.mueller@huggingface.co
License: Apache
Location: c:\Users\Bhavesh\AppData\Local\Programs\Python\Python312\Lib\site-packages
Requires: huggingface-hub, numpy, packaging, psutil, pyyaml, safetensors, torch
Required-by: unstructured-inference
Note: you may need to restart the kernel to use updated packages.


In [1]:
from unstructured.partition.pdf import partition_pdf
from langchain_core.runnables import RunnableLambda,RunnablePassthrough
from langchain.chat_models import ChatOpenAI
from langchain.schema import StrOutputParser
from langchain.prompts import ChatPromptTemplate
import uuid

from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain.storage import InMemoryStore
from langchain_community.vectorstores import Chroma
from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings

In [2]:
raw_pdf_element = partition_pdf(
    filename="./RAG4_MBBS_MM/attention.pdf",
    strategy="hi_res",
    extract_images_in_pdf=True,
    extract_image_block_types=["Image","Table"],
    extract_image_block_to_payload=False,
    extract_image_block_output_dir="./RAG4_MBBS_MM/02/extracted_data/"
)

CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


In [3]:
raw_pdf_element[:5]

In [4]:
Header=[]
Footer=[]
Title=[]
NarrativeText=[]
Text=[]
ListItem=[]

for element in raw_pdf_element:
    
    if "unstructured.documents.elements.Header" in str(type(element)):
        Header.append(str(element))
        
    elif "unstructured.documents.elements.Footer" in str(type(element)):
        Footer.append(str(element))
        
    elif "unstructured.documents.elements.Title" in str(type(element)):
        Title.append(str(element))
        
    elif "unstructured.documents.elements.NarrativeText" in str(type(element)):
        NarrativeText.append(str(element))
        
    elif "unstructured.documents.elements.Text" in str(type(element)):
        Text.append(str(element))
        
    elif "unstructured.documents.elements.ListItem" in str(type(element)):
        ListItem.append(str(element))
        
        


In [5]:
len(Header),len(Footer),len(Title),len(NarrativeText),len(Text),len(ListItem)

(3, 8, 27, 81, 37, 43)

In [6]:
Image = []

for element in raw_pdf_element:
    if "unstructured.documents.elements.Image" in str(type(element)):
        Image.append(str(element))

In [7]:
Table = []

for element in raw_pdf_element:
    if "unstructured.documents.elements.Table" in str(type(element)):
        Table.append(str(element))

In [8]:
len(Image),len(Table)

(7, 4)

In [9]:
Table[0]

'Layer Type Complexity per Layer Sequential Maximum Path Length Operations Self-Attention O(n2 · d) O(1) O(1) Recurrent O(n · d2) O(n) O(n) Convolutional O(k · n · d2) O(1) O(logk(n)) Self-Attention (restricted) O(r · n · d) O(1) O(n/r)'

In [10]:
Image[0]

'Output Probabilities Add & Norm Feed Forward Add & Norm Multi-Head Attention a, Add & Norm Add & Norm Feed Forward Nx | -+CAgc8 Norm) Add & Norm Masked Multi-Head Multi-Head Attention Attention Se a, ee a, Positional Positional Encoding @ © @ Encoding Input Output Embedding Embedding Inputs Outputs (shifted right)'

In [ ]:
# %pip install transformers_stream_generator einops

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for transformers_stream_generator: filename=transformers_stream_generator-0.0.5-py3-none-any.whl size=12549 sha256=5988443140cad5dd7e7e57c3b50f0694bdd4460ad6d0ccd6a25c2668e471b2c7
  Stored in directory: c:\users\bhavesh\appdata\local\pip\cache\wheels\a8\58\d2\014cb67c3cc6def738c1b1635dbf4e3dab6fb63aba7070dce0
Successfully built transformers_stream_generator

   ---------------------------------------- 0/2 [einops]
   ---------------------------------------- 0/2 [einops]
   ---------------------------------------- 2/2 [transformers_stream_generator]

Note: you may need to restart the kernel to use updated packages.


  DEPRECATION: Building 'transformers_stream_generator' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'transformers_stream_generator'. Discussion can be found at https://github.com/pypa/pip/issues/6334

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [15]:
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoProcessor
from PIL import Image

model_path = r"D:\web dev backup\Pytorch\Qwen-VL-Chat"

# Load tokenizer + model
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    device_map="auto",
    torch_dtype="float16",   # saves VRAM
    trust_remote_code=True,
     offload_folder="./RAG4_MBBS_MM//offload"
).eval()

# For multimodal (text + image)
processor = AutoProcessor.from_pretrained(model_path, trust_remote_code=True)

The model is automatically converting to bf16 for faster inference. If you want to disable the automatic precision, please manually add bf16/fp16/fp32=True to "AutoModelForCausalLM.from_pretrained".


Loading checkpoint shards:   0%|          | 0/10 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the disk and cpu.


In [ ]:
# model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen-VL-Chat", ...)
# tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen-VL-Chat")


# Text summary

In [16]:
prompt_text = """You are an assistant tasked with summarizing text for retrieval. 
These summaries will be embedded and used to retrieve the raw text elements. 
Give a concise summary of the table or text that is well optimized for retrieval.text: {element} """

In [18]:
query = "Summarize this medical document"
inputs = tokenizer(query, return_tensors="pt").to(model.device)

outputs = model.generate(**inputs, max_new_tokens=512)
response = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(response)


KeyboardInterrupt: 

In [ ]:
from transformers import pipeline
from langchain.llms import HuggingFacePipeline

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    device=0  # GPU, or -1 for CPU
)

hf_llm = HuggingFacePipeline(pipeline=pipe)


In [ ]:
# Initialize empty summaries
text_summaries = []
text_summaries = summarize_chain.batch(Text, {"max_concurrency": 5})

# Table summary

In [ ]:
prompt_table = """You are an assistant tasked with summarizing table for retrieval. 
These summaries will be embedded and used to retrieve the raw table elements. 
Give a concise summary of the table that is well optimized for retrieval.Table: {element} """

In [ ]:
prompt = ChatPromptTemplate.from_template(prompt_table)

In [ ]:
# table summary chain
model = ChatOpenAI(temperature=0, model="gpt-4")
summarize_chain = {"element": lambda x: x} | prompt | model | StrOutputParser()

In [ ]:
# Initialize empty summaries
table_summaries = []
table_summaries = summarize_chain.batch(Table, {"max_concurrency": 5})

# Image summary


In [ ]:
import base64
import os
from langchain_core.messages import HumanMessage

def encode_image(image_path):
    """Getting the base64 string"""
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")

In [ ]:


def generate_img_summaries(path):
    """
    generate summaries and base64 encoded string for images
    path : path to loist of .jpg files extracted by unstructured
    """
    
    #store base64 encoded images
    img_base64_list= []
    
    # store image summaries
    image_summaries = []
    
    
    #prompt
    
    prompt  = """
    you are assistant talked with summarizing images for retrieval.
    these summaries will be embeded and used to retrieve the raw images.
    give a concise summary of the image that is well optimized for retrieval .
    """
    
    # apply to images
    
    for img_file in sorted(os.listdir(path)):
        
        if img_file.endswith(".jpg"):
            img_path = os.path.join(path,img_file)
            base64_image = encode_image(img_path)
            img_base64_list.append(base64_image)
            image_summaries.append(image_summaries(base64_image,prompt))
        
    
    return img_base64_list,image_summaries

In [ ]:
fpath = "./RAG4_MBBS_MM/"

In [ ]:
img_base64_list,image_summaries = generate_img_summaries(fpath)

In [ ]:
def image_summarize(img_base64, prompt):
    """Make image summary"""
    chat = ChatOpenAI(model="gpt-4-vision-preview", max_tokens=1024)

    msg = chat.invoke(
        HumanMessage(
            content=[
                {"type": "text", "text": prompt},
                {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{img_base64}"},
                },
            ]
        )
    )
    return msg.content

In [ ]:
def create_multi_vector_retriever(vectorstore,text_summaries,texts,tables_summaries,tables,image_summaries,images):
    
    """
    create retriever that indexes summarieses , but returns raw image or texts
    """
    
    # initialization
    
    store = InMemoryStore()
    id_key='doc_id'
    
    
    #create the mmutli-vector retriever
    
    retriever = MultiVectorRetriever(
        vectorstore=vectorstore,
        docstore=store,
        id_key=id_key
    )
    
    
    def add_documents(retriever,doc_summaries,doc_contents):
        
        doc_ids = [str(uuid.uuid4()) for _ in doc_contents ]
        
        summary_docs = [
            Document(page_contents = s ,metadata={id_key:doc_ids[i]})
            for i,s in enumerate(doc_summaries)
        ]
        
        retriever.vectorstore.add_documents(summary_docs)
        retriever.docstore.mset(list(zip(doc_ids,doc_contents)))
        
        
        #add text tables images/
        #check that text summaries is not emty before adding 
        
        if text_summaries:
            add_documents(retriever,text_summaries,texts)
            
        # check for table too
        
        if tables_summaries:
            add_documents(retriever,tables_summaries,tables)
            
        if image_summaries:
            add_documents(retriever,image_summaries,images)
            
            
        return retriever
    
vectorstore = Chroma(
    collection_name = "mm_rag",embedding_function=OpenAIEmbeddings()
)

## create retriever

retriever_multi_vector_img = create_multi_vector_retriever(
    vectorstore,
    text_summaries,
    Text,
    table_summaries,
    Table,
    image_summaries,
    img_base64_list,
    
)

In [ ]:
retriever_multi_vector_img

In [ ]:
import io,re
from IPython.display import HTML,display
from PIL import Image


In [ ]:
def plt_img_base64(img_base64):
    
    image_html = f'<img src="data:image/jpeg;base64",{img_base64} />'
    
    display(HTML(image_html))
    
plt_img_base64(img_base64_list[1])

In [ ]:
def looks_like_base64(sb):
    """
    check if the string looks like base64
    """
    
    return re.match("^[A-Za-z0-9+/]+[=]{0,2}$",sb) is not None

In [ ]:
def is_image_data(b64data):
    """
    Check if the base64 data is an image by looking at the start of the data
    """
    image_signatures = {
        b"\xFF\xD8\xFF": "jpg",
        b"\x89\x50\x4E\x47\x0D\x0A\x1A\x0A": "png",
        b"\x47\x49\x46\x38": "gif",
        b"\x52\x49\x46\x46": "webp",
    }
    try:
        header = base64.b64decode(b64data)[:8]  # Decode and get the first 8 bytes
        for sig, format in image_signatures.items():
            if header.startswith(sig):
                return True
        return False
    except Exception:
        return False

In [ ]:
def resize_base64_image(base64_string,size=(128,128)):
    """
    Resize an image encoded as base64 string
    """
    
    # decode the base64 string
    
    img_data = base64.b64decode(base64_string)
    img= Image.open(io.BytesIO(img_data))
    
    #resize images
    resized_img = img.resize(size,Image.LANCZOS)
    
    # save the resized image to a bytes buffer
    buffered = io.BytesIO()
    resized_img.save(buffered,format=img.format)
    
    
    # encode the resized image to base64
    return base64.b64encode(buffered.getvalue()).decode("utf-8")

In [ ]:
def split_image_text_types(docs):
    """
    Split base64-encoded images and texts
    """
    
    b64_images = []
    texts = []
    
    for doc in docs:
        
        #check if the document(doc) is of the type document and extract page_content if sorted
        if isinstance(doc,Document):
            doc = doc.page_content
            
        if looks_like_base64(doc) and is_image_data(doc):
            
            doc = resize_base64_image(doc,size=(1300,600))
            b64_images.append(doc)
            
        else:
            texts.append(doc)
            
    print(b64_images)
    print(texts)
        
    return {"images":b64_images ,
                "texts":texts
                }

In [ ]:
def img_prompt_func(data_dict):
    """
    Join the context into a single string
    """
    # print(data_dict)
    formatted_texts = "\n".join(data_dict["context"]["texts"])
    messages = []

    # Adding image(s) to the messages if present
    if data_dict["context"]["images"]:
        for image in data_dict["context"]["images"]:
            image_message = {
                "type": "image_url",
                "image_url": {"url": f"data:image/jpeg;base64,{image}"},
            }
            messages.append(image_message)

    # Adding the text for analysis
    text_message = {
        "type": "text",
        "text": (
            "You are a helpful assistant.\n"
            "You will be given a mixed info(s).\n"
            "Use this information to provide relevant information to the user question.\n"
            f"User-provided question: {data_dict['question']}\n\n"
            "Text and/or tables:\n"
            f"{formatted_texts}"
        ),
    }
    messages.append(text_message)

    return [HumanMessage(content=messages)]

In [ ]:
from langchain_core.runnables import RunnableLambda,RunnablePassthrough
from langchain.chat_models import ChatOpenAI
from langchain.schema import StrOutputParser


def multi_model_rag_chain(retriever):
    """
    Multi model RAG chain
    """
    
    model = ChatOpenAI(temperature=0,model="gpt-5-vision-preview",max_tokens=1024)
    
    
    # RAG pipeline
    
    chain = (
        {
            "context": retriever | RunnableLambda(split_image_text_types),
            "question": RunnablePassthrough(),
            
        }
        | RunnableLambda(img_prompt_func)
        | model
        | StrOutputParser()
        
    )
    
    return chain

In [ ]:
# create RAG chain

chain_multi_model_rag = multi_model_rag_chain(retriever_multi_vector_img)

In [ ]:
chain_multi_model_rag

In [ ]:
query = "whats the paper about ?"

In [ ]:
chain_multi_model_rag.invoke(query)